<a href="https://colab.research.google.com/github/dasdasqs2/Machine-learning/blob/main/HW/HW2_20183269_kimhyunjung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW2
6주차 강의노트에서 diabetes 데이터를 다루는 예제가 있다.


1. 이 예제에서 출력하는 값은 loss 밖에 없다. 
   loss 가 아닌 accuracy 를 출력하게 프로그램을 수정하라.

2. 이 예제에서는 데이터를 모두 학습하는데 사용한다.
   데이터를 random하게 학습데이터와 테스트데이터를 8:2로 나누고,
   학습할 때, 학습데이터 중 25%를 validation 데이터로 사용게끔 프로그램을 수정하라

3. 문제2의 결과를 accuracy로 보여라

## Method 1

In [177]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

tf.random.set_seed(42)

In [178]:
xy = np.loadtxt('diabetes_data.csv', delimiter=',', dtype=np.float32)
x = xy[:, 0:-1]
y = xy[:,[-1]]

# train:test = 8:2
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=1234)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# validation = 25%
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, shuffle=True, random_state=1234)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)


(607, 8) (607, 1) (152, 8) (152, 1)
(455, 8) (152, 8) (455, 1) (152, 1)


In [179]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))

In [180]:
def logistic_regression(features): # features : x_train
  # hypothesis = tf.divide(1., 1. + tf.exp(-tf.matmul(features, W) -b)) # 또는 tf.sigmoid(z)
  z = tf.matmul(features, W) + b
  hypothesis = tf.sigmoid(z)
  return hypothesis

In [181]:
def cost_fn(hypothesis, feature, y):
  cost = -tf.reduce_mean(y * tf.math.log(hypothesis) + (1-y) * tf.math.log(1-hypothesis))
  return cost

In [182]:
def loss_fn(hypothesis, features, labels):
  cost = -tf.reduce_mean(labels + tf.math.log(logistic_regression(features)) + (1-labels) * tf.math.log(1-hypothesis))
  return cost

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [183]:
def accuracy_fn(hypothesis, labels):
  predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.int32))
  return accuracy

In [184]:
# training

EPOCHS = 1001

tf.random.set_seed(42)

W = tf.Variable(tf.random.normal((8,1)), name='weight')
b = tf.Variable(tf.random.normal((1,)), name='bias')

optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

for step in range(EPOCHS):
  for features, labels in iter(dataset):
    with tf.GradientTape() as tape:
      loss_value = cost_fn(logistic_regression(features), features, labels)

    grads = tape.gradient(loss_value, [W,b])
    optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))

    if step % 100 == 0:
      # print("W: {}, b: {}, Iter: {}, Loss: {:.4f}".format(W.numpy(), b.numpy(), step, cost_fn(logistic_regression(features),features,labels)))
      print("W: {}, b: {}, Iter: {}, Accuracy: {:.4f}".format(W.numpy(), b.numpy(), step, accuracy_fn(logistic_regression(features),labels)))

W: [[ 0.327345  ]
 [-0.84323144]
 [ 0.31923434]
 [-1.4073657 ]
 [-2.387585  ]
 [-1.0394387 ]
 [-0.55706614]
 [ 0.5397554 ]], b: [0.08367751], Iter: 0, Accuracy: 0.0000
W: [[ 0.3121567 ]
 [-0.9012761 ]
 [ 0.30085483]
 [-1.390479  ]
 [-2.341876  ]
 [-1.0585222 ]
 [-0.5364719 ]
 [ 0.540933  ]], b: [0.03667465], Iter: 100, Accuracy: 0.0000
W: [[ 0.29258046]
 [-0.95510286]
 [ 0.28503928]
 [-1.3764402 ]
 [-2.2992303 ]
 [-1.0775433 ]
 [-0.52404463]
 [ 0.5363591 ]], b: [0.00206796], Iter: 200, Accuracy: 0.0000
W: [[ 0.2701016 ]
 [-1.0056335 ]
 [ 0.27116466]
 [-1.3645062 ]
 [-2.2590075 ]
 [-1.0964267 ]
 [-0.51741326]
 [ 0.52784467]], b: [-0.02359774], Iter: 300, Accuracy: 0.0000
W: [[ 0.24580432]
 [-1.0535442 ]
 [ 0.2587708 ]
 [-1.3541172 ]
 [-2.2207026 ]
 [-1.1151181 ]
 [-0.5148391 ]
 [ 0.5167144 ]], b: [-0.0428588], Iter: 400, Accuracy: 0.0000
W: [[ 0.22046164]
 [-1.0993268 ]
 [ 0.24752392]
 [-1.3448611 ]
 [-2.1839378 ]
 [-1.1335787 ]
 [-0.51507205]
 [ 0.50391495]], b: [-0.05754043], Iter: 50

In [185]:
# print test accuracy

test_acc = accuracy_fn(logistic_regression(x_test), y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.0000


## Method 2

In [186]:
tf.random.set_seed(42)
xy = np.loadtxt('diabetes_data.csv', delimiter=',', dtype=np.float32)

x = xy[:, 0:-1]
y = xy[:,[-1]]

# train:test = 8:2
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=1234)
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# validation = 25%
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, shuffle=True, random_state=1234)
print(x_train.shape, x_val.shape, y_train.shape, y_val.shape)

(607, 8) (607, 1) (152, 8) (152, 1)
(455, 8) (152, 8) (455, 1) (152, 1)


In [187]:
model = LogisticRegression()

In [188]:
model.fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### Validation

In [189]:
y_pred_val = model.predict(x_val)
y_pred_val

array([0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1.],
      dtype=float32)

In [190]:
# print accuracy
tf.random.set_seed(42)
accuracy = metrics.accuracy_score(y_val, y_pred_val)
accuracy_percentage = 100 * accuracy
print("Accuracy :", accuracy_percentage)

Accuracy : 73.02631578947368


### Test

In [191]:
tf.random.set_seed(42)
y_pred_test = model.predict(x_test)
y_pred_test

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0.,
       1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1.,
       1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.],
      dtype=float32)

In [192]:
# print accuracy
tf.random.set_seed(42)
accuracy = metrics.accuracy_score(y_test, y_pred_test)
accuracy_percentage = 100 * accuracy
print("Accuracy :", accuracy_percentage)

Accuracy : 75.0
